In [1]:
import pandas as pd
import numpy as np

# Bokeh es la biblioteca que usaremos para las gráficas
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, HoverTool, NumeralTickFormatter, DataTable, TableColumn, NumberFormatter
from bokeh.palettes import Dark2, YlGn9, Viridis, Pastel1

# output_notebook() activa el despliegue de gráficas en un notebook.
output_notebook()

Loading BokehJS ...

In [2]:
df = pd.read_csv("answers-2022.csv", index_col=0)
# El salario de las personas de Mx se guarda en la columna salarymx y está en pesos.
# En la mayoría de los casos solo vamos a tomar en cuenta los datos de personas en Mx con salario en pesos.
df = df[(df['country'] == 'México')]
df.count()

created            1921
recaptcha_score      24
salarymx           1921
salaryusd          1921
extramx            1921
                   ... 
covid_salario        19
covid_carga          19
covid_apoyo        1419
vacaciones         1502
aguinaldo          1510
Length: 198, dtype: int64

In [3]:
df.salarymx.describe()

count      1921.000000
mean      54703.730349
std       40111.310102
min           0.000000
25%       27000.000000
50%       46000.000000
75%       70000.000000
max      350000.000000
Name: salarymx, dtype: float64

In [4]:
df.experience.describe()

count    1921.000000
mean       10.517959
std         7.693261
min         0.000000
25%         5.000000
50%         9.000000
75%        15.000000
max        40.000000
Name: experience, dtype: float64

## Experiencia

In [5]:
df['exp_bin'] = pd.cut(df['experience'],bins=[-0.1,2,4,6,8,10,14,20,40])

exp = df.groupby('exp_bin')['salarymx'].agg(['count', 'median','mean', 'std'])
exp['std'] = round(exp['std']).astype(int)
# Groupby deja exp_bin como un índice, lo necesitamos como una columna normal así que damos reset_index.
exp = exp.reset_index()
exp.head(10)

,exp_bin,count,median,mean,std
0,"(-0.1, 2.0]",217,23000.0,26904.493088,19109
1,"(2.0, 4.0]",259,30000.0,35652.992278,22135
2,"(4.0, 6.0]",251,39000.0,45515.633466,31827
3,"(6.0, 8.0]",189,50000.0,56527.111111,37296
4,"(8.0, 10.0]",228,55000.0,62145.144737,42384
5,"(10.0, 14.0]",265,62000.0,66078.675472,32538
6,"(14.0, 20.0]",301,60000.0,66535.378738,41048
7,"(20.0, 40.0]",211,60000.0,76769.322275,57777


In [6]:

# Los valores en exp_bin son intervalos pero para la gráfica necesitamos que sean strings/categoricas.
exp_labels = ['0-2', '3-4', '5-6', '7-8', '9-10', '11-14', '15-20', '20+']
exp['exp_bin'] = exp_labels

# Tomo la paleta YlGn9 que tiene 9 colores y uso los primeros 8. No uso YlGn8 porque el último tono es muy claro.
# Pongo el resultado en una variable aux para luego poder invertir el orden de colores (más claro al principio)
aux = YlGn9[0:8]
exp['color'] = aux[::-1]

src = ColumnDataSource(exp)
p = figure(x_range=exp_labels, plot_height=400, plot_width=700)
p.vbar(source=src, x='exp_bin', top='median', width=0.95, color='color')
p.title.text = 'Salario medio de acuerdo a a la experiencia'
p.xaxis.axis_label = 'Experiencia (años)'
p.yaxis.axis_label = 'Salario bruto mensual (MXN)'
p.yaxis.formatter = NumeralTickFormatter(format='$0 a')

hover = HoverTool()
hover.tooltips=[
    ('Experiencia', '@exp_bin años'),
    ('Observaciones', '@count'),
    ('Salario medio', '@median{$0,0}'),
]
p.add_tools(hover)

show(p)

## Comparación por género

In [7]:
# Agrupamos por experiencia y género. Llenamos con 0 los grupos sin valores.
gender = df.groupby(['exp_bin', 'gender'])['salarymx'].agg(['median','count']).fillna(0)
gender.head(25)

median  count
exp_bin      gender                 
(-0.1, 2.0]  hombre   23000.0    156
             mujer    20038.0     56
             nb       25500.0      5
(2.0, 4.0]   hombre   32000.0    189
             mujer    30000.0     67
             nb       23500.0      3
(4.0, 6.0]   hombre   40000.0    192
             mujer    35000.0     57
             nb       23500.0      2
(6.0, 8.0]   hombre   52000.0    141
             mujer    37550.0     48
             nb           0.0      0
(8.0, 10.0]  hombre   60000.0    187
             mujer    35000.0     41
             nb           0.0      0
(10.0, 14.0] hombre   65000.0    219
             mujer    46000.0     44
             nb       55500.0      2
(14.0, 20.0] hombre   60000.0    258
             mujer    49000.0     42
             nb       56000.0      1
(20.0, 40.0] hombre   67000.0    185
             mujer    41000.0     24
             nb      148000.0      2

In [8]:
# El dataframe que arroja el groupby no se presta a graficar, así que lo reacomodamos en uno nuevo.
# El número de observaciones de género no binario es muy bajo y no arroja datos robustos así que lo omitiré.
data = {
    'exp': exp_labels,
    'hombre_salary' : list(gender.xs('hombre',level=1)['median']),
    'hombre_count' : list(gender.xs('hombre',level=1)['count']),
    'mujer_salary' : list(gender.xs('mujer',level=1)['median']),
    'mujer_count' : list(gender.xs('mujer',level=1)['count'])
}
genderdf = pd.DataFrame(data)
genderdf.head(10)

,exp,hombre_salary,hombre_count,mujer_salary,mujer_count
0,0-2,23000.0,156,20038.0,56
1,3-4,32000.0,189,30000.0,67
2,5-6,40000.0,192,35000.0,57
3,7-8,52000.0,141,37550.0,48
4,9-10,60000.0,187,35000.0,41
5,11-14,65000.0,219,46000.0,44
6,15-20,60000.0,258,49000.0,42
7,20+,67000.0,185,41000.0,24


In [9]:
# Teniendo esta estructura más amigable procedamos a generar lineas con sus tooltips.
src = ColumnDataSource(genderdf)
p = figure(x_range=exp_labels, plot_height=400)

renderer = p.line(x='exp',y='hombre_salary', source=src, color ='#1f77b4', line_width=2, legend_label='hombre')
p.add_tools(HoverTool(
            renderers=[renderer],
            tooltips=[
                ('Género', 'Hombre'),
                ('Experiencia', '@exp años'),
                ('Observaciones', '@hombre_count'),
                ('Salario', '@hombre_salary{$0,0}')
            ]
        ))

renderer = p.line(x='exp',y='mujer_salary', source=src, color ='#e617e6', line_width=2, legend_label='mujer')
p.add_tools(HoverTool(
            renderers=[renderer],
            tooltips=[
                ('Género', 'Mujer'),
                ('Experiencia', '@exp años'),
                ('Observaciones', '@mujer_count'),
                ('Salario', '@mujer_salary{$0,0}')
            ]
        ))


p.title.text = 'Comparativo por género'
p.xaxis.axis_label = 'Experiencia (años)'
p.yaxis.axis_label = 'Salario bruto mensual (MXN)'
p.yaxis.formatter = NumeralTickFormatter(format='$0 a')
p.legend.location = "bottom_right"

show(p)


In [10]:

gender = df[(df['experience'] > 0)].groupby(["experience", "gender"])['salarymx'].agg(['median','count'])
gender = gender.reset_index()
gender = gender[(gender['count'] > 4)]
gender['size'] = round(np.sqrt(gender['count']))*2

# Agregamos una columna de color en base al valor de gender.
gender['color'] = ['#1f77b4' if x =='hombre' else '#e617e6' for x in gender['gender']] 

src = ColumnDataSource(gender)
p = figure(plot_height = 400, plot_width = 600, sizing_mode="scale_both", toolbar_location=None)
p.toolbar.active_drag = None
p.toolbar.active_scroll = None
p.circle(x='experience', y='median', source=src, size='size', color='color')
p.title.text = 'Comparativo por género'
p.xaxis.axis_label = 'Experiencia (años)'
p.yaxis.axis_label = 'Salario bruto mensual (MXN)'
p.yaxis.formatter = NumeralTickFormatter(format='$0 a')

hover = HoverTool()
hover.tooltips=[
        ('Género', '@gender'),    
        ('Experiencia', '@experience años'),
        ('Observaciones', '@count'),
        ('Salario medio', '@median{$0,0}'),
    ]

p.add_tools(hover)
show(p)

In [11]:
gx = pd.crosstab(index=df['city'], columns=df['gender'])
gx = gx[(gx['hombre'])>9]
gx

gender,hombre,mujer,nb
city,,,
Aguascalientes,41,6,0
CDMX,36,9,0
Cancún,13,2,0
Chihuahua,22,5,0
Ciudad de México,18,7,0
Colima,14,5,1
Cuernavaca,12,4,0
Culiacán,16,3,0
Estado de México,13,4,0


## Inglés

In [12]:
ingles = df.groupby("english_label")["salarymx"].agg(['count', 'median', 'mean', 'std']).sort_values(by=['median'], ascending=False)
ingles.head(20)

,count,median,mean,std
english_label,,,,
Nativo o bilingue (ILR 5),74,75000.0,87328.432432,55881.106616
Avanzado: Puedo conversar y escribir sin problemas sobre cualquier tema (ILR 4),456,66500.0,74711.763158,50959.370667
Profesional: Puedo interactuar profesionalmente con colegas y clientes (ILR 3),627,52000.0,56802.569378,33979.180394
Limitado: Me doy a entender pero con errores de gramática (ILR 2),539,35000.0,40361.647495,26565.998723
Elemental: Sé lo básico para sobrevivir (ILR 1),208,30000.0,32501.725962,19701.293922
Ninguno (ILR 0),17,18000.0,24970.588235,16362.906401


In [13]:
ingles = df.groupby(['english_num','exp_bin'])['salarymx'].agg(['median', 'count']).fillna(0)
data = {
    'exp': exp_labels,
    'Elemental': list(ingles.xs(1)['median']),
    'Limitado' : list(ingles.xs(2)['median']),
    'Profesional' : list(ingles.xs(3)['median']),
    'Avanzado' : list(ingles.xs(4)['median']),
    'Nativo' : list(ingles.xs(5)['median'])    
}

inglesdf = pd.DataFrame(data)

src = ColumnDataSource(inglesdf)

p = figure(x_range=exp_labels, plot_height=400, plot_width=600)

for col_name, color in zip(list(inglesdf.columns), Dark2[6]):
    if col_name == 'exp':
        continue

    p.add_tools(HoverTool(
        renderers= [p.line('exp', col_name, source=src, color=color, legend_label=col_name, line_width=2)],
        tooltips=[
            ('Ingles', col_name),
            ('Salario', '@'+col_name+'{$0,0}')
        ]
    ))

p.title.text = 'Salario de acuerdo al nivel de inglés'
p.xaxis.axis_label = 'Experiencia (años)'
p.yaxis.axis_label = 'Salario'
p.yaxis.formatter = NumeralTickFormatter(format='$0 a')
p.legend.location = "top_left"

    
show(p)


## Agrupación por ciudad y país

In [14]:
# Solo tomamos en cuenta a las personas con perfil de empleado (no freelancers, directivos ni emprendedores)
# Incluimos la experiencia como variable observada para obtener el promedio de experiencia por ciudad
cities = df[(df["profile"] == "godin")].groupby("city")['salarymx', 'experience'].agg(['count', 'median', 'mean', 'std'])
cities = cities.reset_index()

# El group by de múltiples columnas observadas con múltiples funciones agregadas nos genera que los nombre son tuplas
# así que hacemos este map para renombrar las columnas.
cities.columns = cities.columns.map('_'.join)

cities = cities[(cities["salarymx_count"]> 10)]
cities = cities[(cities["salarymx_median"]> 0)]
cities = cities.sort_values(by=['salarymx_median'], ascending=False)
src = ColumnDataSource(cities)
columns = [
        TableColumn(field="city_", title="Ciudad"),
        TableColumn(field="salarymx_count", title="n"),
        TableColumn(field="salarymx_median", title="Mediana", formatter=NumberFormatter(format='$0,0')),
        TableColumn(field="salarymx_mean", title="Media", formatter=NumberFormatter(format='$0,0')),
        TableColumn(field="salarymx_std", title="Des std", formatter=NumberFormatter(format='$0,0')),
        TableColumn(field="experience_mean", title="Experiencia promedio", formatter=NumberFormatter(format='0')),
    ]
table = DataTable(source=src,columns=columns,index_position=None, width=400)
show(table)

/var/folders/mt/4vyh28v93sn_p_vdvr053kxc0000gn/T/ipykernel_89143/2327758837.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  cities = df[(df["profile"] == "godin")].groupby("city")['salarymx', 'experience'].agg(['count', 'median', 'mean', 'std'])


In [15]:
#guardamos en un csv para importar a sheets
cities.to_csv("tablas/cities.csv")

Hagamos el breakdown por país. Para ello nos basamos en la columna salaryusd.

In [16]:
# Volvemos a leer del csv para incluir datos de otros países
df2 = pd.read_csv("answers-2021.csv", index_col=0)
df2['salaryusd'] = np.where((df2.salaryusd == 0),round(df2.salarymx/19,0).astype(int), df2.salaryusd)
countries = df2.groupby("country")["salaryusd"].agg(['count', 'median', 'mean', 'std'])
countries = countries.reset_index()
countries = countries.sort_values(by=['median'], ascending=False)
countries.head(30)

,country,count,median,mean,std
6,Estados Unidos,18,11116.5,11411.111111,6093.012178
1,Canada,3,9000.0,10500.000000,3968.626967
4,Ecuador,5,2240.0,1902.000000,840.606924
5,España,2,2150.0,2150.000000,212.132034
8,México,1526,2105.0,2486.761468,1811.436312
2,Chile,19,2034.0,2355.578947,2093.359536
9,Otro,9,2000.0,5265.444444,7872.356336
7,Guatemala,2,1912.5,1912.500000,159.099026
10,Perú,13,1800.0,1913.923077,1080.276235
3,Colombia,14,1025.0,1274.071429,650.988061


## Lenguajes

El dilema con los lenguajes es que cada participante puede escoger varios lenguajes (máximo 3) que utiliza. Así que no podemos tener una única variable categórica para lenguaje, sino que tenemos una variable booleana (Y/N) por cada una de las opciones de lenguaje. Si tuvieramos una única variable, simplemente podríamos hacer un groupby y listo, pero al no tenerla, tenemos que "armar" nuestro dataframe.

Vamos a generar una lista donde cada elemento es a su vez una lista con la info de cada lenguaje (nombre del lenguaje, número de observaciones y salario medio). A partir de esa lista de listas generamos un dataframe y continuamos como de costumbre.

In [17]:
# Inicializamos nuestra lista maestra
lang_list = []
# Leemos la lista de lenguajes a partir de un catálogo.
import csv
with open("lang_options.csv") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        lang_key = "lang_"+row['key']
        # Creamos una lista con el nombre del lenguaje, su num. de observaciones y salario medio, y agregamos dicha lista a nuestra lista maestra.
        lang_list.append([row['name'], df[(df[lang_key]=="Y")]["salarymx"].count(), df[(df[lang_key]=="Y")]["salarymx"].median(), df[(df[lang_key]=="Y")]["salarymx"].mean(), df[(df[lang_key]=="Y")]["salarymx"].std(), df[(df[lang_key]=="Y")]["experience"].mean()])

# Una vez que tenemos la lista maestra completa, creamos un dataframe indicando el nombre de las columnas.
langdf = pd.DataFrame(lang_list, columns = ['lenguaje', 'n', 'mediana','media', 'std', 'exp'])
# Agregamos una columna de popularidad que se calcule en base a un valor amortiguado del número de observaciones.        
langdf['popularidad'] = round(np.sqrt(langdf['n']/2)*2)
langdf.sort_values(by=['mediana'], ascending=False).head(40)





,lenguaje,n,mediana,media,std,exp,popularidad
5,Elixir,13,70000.0,95961.538462,65819.975925,8.846154,5.0
15,Ruby,56,64000.0,71579.464286,45504.846672,7.589286,11.0
11,Perl,4,60000.0,59000.000000,6633.249581,12.500000,3.0
7,Groovy,22,60000.0,71901.727273,42407.248807,12.000000,7.0
0,Bash,67,58000.0,64847.805970,52719.150161,10.447761,12.0
6,Go,35,53000.0,65000.000000,36849.455260,8.200000,8.0
8,Java,293,50000.0,60769.593857,45755.429397,9.764505,24.0
10,Kotlin,19,50000.0,53968.421053,31651.786901,8.210526,6.0
3,COBOL,7,47300.0,44942.857143,8884.041337,11.857143,4.0
17,Scala,8,46000.0,72850.000000,49325.826327,6.500000,4.0


In [18]:
# Esto exporta el dataframe a un csv
langdf.to_csv("tablas/langs.csv")


In [19]:
from bokeh.models import LabelSet
from bokeh.models import Range1d

src = ColumnDataSource(langdf)
p = figure()
# Ponemos los circulos invisibles pero con tamaño para que sirvan los tooltips en hover.
p.circle(source=src, y='popularidad', x='mediana', line_color=None, fill_color=None, size=20)
p.yaxis.axis_label = 'Popularidad'

# La escala de popularidad tiene unidades arbitrarias así que prefiero evitar que se despliegue.
p.yaxis.major_label_text_font_size = '0pt'

p.xaxis.axis_label = 'Salario bruto mensual (MXN)'
p.xaxis.formatter = NumeralTickFormatter(format='$0 a')    
p.x_range = Range1d(10000, 70000)

labels = LabelSet(source=src, x='mediana', y='popularidad', text='lenguaje', level='glyph',
              x_offset=-10, y_offset=-5, render_mode='canvas', text_font_size="9pt", text_color='#1f77b4')
p.add_layout(labels)

hover = HoverTool()
hover.tooltips=[
    ('Lenguaje', '@lenguaje'),
    ('Observaciones', '@count'),
    ('Salario medio', '@salario{$0,0}'),
]

p.add_tools(hover)

show(p)

## Front end

In [20]:
# Inicializamos nuestra lista maestra
item_list = []
# Leemos la lista de opciones a partir de un catálogo.
import csv
with open("front_options.csv") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        item_key = "front_"+row['key']
        # Por cada elemento en la lista agregamos un renglon con el nombre de la categoria y estadisticas agregadas.
        item_list.append([row['name'], df[(df[item_key]=="Y")]["salarymx"].count(), df[(df[item_key]=="Y")]["salarymx"].median(), df[(df[item_key]=="Y")]["salarymx"].mean(), df[(df[item_key]=="Y")]["salarymx"].std(), df[(df[item_key]=="Y")]["experience"].mean()])

# Una vez que tenemos la lista maestra completa, creamos un dataframe indicando el nombre de las columnas.
itemdf = pd.DataFrame(item_list, columns = ['Tecnologia front', 'n', 'mediana','media', 'std', 'exp'])
# Agregamos una columna de popularidad que se calcule en base a un valor amortiguado del número de observaciones.        
itemdf.sort_values(by=['mediana'], ascending=False).head(30)

,Tecnologia front,n,mediana,media,std,exp
5,Ruby on Rails,56,53000.0,53428.982143,31812.503417,6.892857
3,Next.js,66,49000.0,50277.893939,30040.394023,7.469697
4,React,301,44000.0,51154.292359,35649.863135,6.953488
6,Semantic UI,4,44000.0,45125.000000,25470.162282,12.250000
7,Spring Boot,110,41250.0,48980.836364,33182.461892,8.990909
0,Angular,226,38000.0,43919.623894,27439.426093,8.429204
1,Django,45,35000.0,43714.177778,28353.894954,7.266667
9,Vue.js,118,30000.0,36041.330508,24268.126831,6.686441
8,svelte,5,27500.0,40900.000000,21605.554841,6.600000
2,Laravel,73,23500.0,28276.123288,17103.753906,7.534247


In [21]:
itemdf.to_csv("tablas/front.csv")

## Certificaciones

In [22]:
# Inicializamos nuestra lista maestra
item_list = []
# Leemos la lista de opciones a partir de un catálogo.
import csv
with open("cert_options.csv") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        item_key = "cert_"+row['key']
        # Por cada elemento en la lista agregamos un renglon con el nombre de la categoria y estadisticas agregadas.
        item_list.append([row['name'], df[(df[item_key]=="Y")]["salarymx"].count(), df[(df[item_key]=="Y")]["salarymx"].median(), df[(df[item_key]=="Y")]["salarymx"].mean(), df[(df[item_key]=="Y")]["salarymx"].std(), df[(df[item_key]=="Y")]["experience"].mean()])

# Una vez que tenemos la lista maestra completa, creamos un dataframe indicando el nombre de las columnas.
itemdf = pd.DataFrame(item_list, columns = ['Certificacion', 'n', 'mediana','media', 'std', 'exp'])
# Agregamos una columna de popularidad que se calcule en base a un valor amortiguado del número de observaciones.        
itemdf.sort_values(by=['mediana'], ascending=False).head(30)

,Certificacion,n,mediana,media,std,exp
8,Enterprise Governance (CGEIT),4,140000.0,147500.000000,92150.239645,22.250000
7,"Enterprise Architect (SEI, IASA, Togaf, Zachman)",28,80300.0,109317.500000,64249.727345,19.178571
25,Testing Advanced (CTAL),9,70000.0,104888.888889,92404.605465,13.888889
19,PMP,61,64500.0,72068.852459,43499.664907,19.704918
20,SAP (cualquier módulo),30,63000.0,75545.000000,72613.830199,15.633333
2,AWS Developer,33,61000.0,62419.727273,28605.930798,11.272727
0,Agile Certified Practitioner,108,60000.0,67882.185185,50343.763725,12.379630
13,ITIL Practitioner,60,60000.0,71766.666667,43782.380430,18.666667
3,AWS Solution Architect,60,60000.0,60850.000000,26721.498359,10.683333
14,Java,166,57500.0,64743.427711,45485.945185,11.951807


In [23]:
itemdf.to_csv("tablas/certs.csv")

## Infraestructura

In [24]:
# Inicializamos nuestra lista maestra
item_list = []
# Leemos la lista de opciones a partir de un catálogo.
import csv
with open("infra_options.csv") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        item_key = "infra_"+row['key']
        # Por cada elemento en la lista agregamos un renglon con el nombre de la categoria y estadisticas agregadas.
        item_list.append([row['name'], df[(df[item_key]=="Y")]["salarymx"].count(), df[(df[item_key]=="Y")]["salarymx"].median(), df[(df[item_key]=="Y")]["salarymx"].mean(), df[(df[item_key]=="Y")]["salarymx"].std(), df[(df[item_key]=="Y")]["experience"].mean()])

# Una vez que tenemos la lista maestra completa, creamos un dataframe indicando el nombre de las columnas.
itemdf = pd.DataFrame(item_list, columns = ['Infra', 'n', 'mediana','media', 'std', 'exp'])
# Agregamos una columna de popularidad que se calcule en base a un valor amortiguado del número de observaciones.        
itemdf.sort_values(by=['mediana'], ascending=False).head(30)

,Infra,n,mediana,media,std,exp
2,Consul,2,96000.0,96000.000000,50911.688245,5.500000
13,Spinnaker,4,91000.0,84750.000000,27705.294320,8.750000
3,Datadog,14,89000.0,99214.285714,79378.527005,8.428571
1,Chef,3,80000.0,88666.666667,23245.071162,11.666667
12,Puppet,2,77500.0,77500.000000,53033.008589,10.000000
8,Kubernetes,63,71500.0,74735.333333,45921.450892,10.936508
0,Ansible,36,62500.0,68966.666667,30240.606570,10.500000
9,New Relic,4,61000.0,68500.000000,22649.503306,8.000000
5,Dynatrace,6,60500.0,57583.333333,25389.794538,9.833333
4,Docker,90,60000.0,62007.033333,29040.175575,10.222222


## Actividad

In [25]:
# Inicializamos nuestra lista maestra
master_list = []
# Leemos la lista de lenguajes a partir de un catálogo.
import csv
with open("activity_options.csv") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        key = "act_"+row['key']
        master_list.append([row['name'], df[(df[key]=="Y")]["salarymx"].count(), df[(df[key]=="Y")]["salarymx"].median(), df[(df[key]=="Y")]["salarymx"].mean(), df[(df[key]=="Y")]["salarymx"].std()])

# Una vez que tenemos la lista maestra completa, creamos un dataframe indicando el nombre de las columnas.
act_df = pd.DataFrame(master_list, columns = ['actividad', 'n', 'mediana', 'mean', 'std'])
act_df.sort_values(by=['mediana'], ascending=False).head(30)

,actividad,n,mediana,mean,std
12,Dirección / Estrategia,171,75000.0,88204.912281,59433.406750
20,Preventa / Tech sales,40,75000.0,80755.000000,51986.028448
6,Coaching y mejora de procesos,92,61500.0,70492.282609,42178.040823
2,Arquitectura y diseño de sistemas,454,60000.0,68716.687225,44444.179687
18,Project management / Coordinación,271,56000.0,63436.653137,38648.947706
11,Dev relations,14,55500.0,56285.714286,36144.323039
15,"Gestión de infraestructura (SysOps, DevOps)",153,55000.0,60935.509804,38001.680546
25,Venta y desarrollo de negocios,40,55000.0,71765.000000,55119.392790
7,Consultoría de negocio,83,55000.0,66076.951807,45851.381044
17,Ingeniería de datos,89,53000.0,59258.685393,34318.876186


In [26]:
act_df.to_csv("tablas/actividad.csv")

## Estudios

In [27]:
educacion = df.groupby("education")["salarymx"].agg(['count', 'median', 'mean', 'std']).sort_values(by=['median'], ascending=False)
educacion['mean'] = round(educacion['mean']).astype(int)
educacion

,count,median,mean,std
education,,,,
maestria,318,56500.0,64171,42859.488141
phd,30,53000.0,62570,35706.206840
posgrado,54,45500.0,68587,61970.616800
universidad,971,45000.0,52989,37599.993850
tecnica,30,41500.0,48864,31168.819016
pasante,481,40000.0,50292,39658.994466
prepa,35,38000.0,55250,40649.110075
secundaria,2,28000.0,28000,14142.135624


## ¿Dónde aprendiste a programar?

In [28]:
edutype = df.groupby("edutype")["salarymx"].agg(['count', 'median', 'mean', 'std']).sort_values(by=['median'], ascending=False)
edutype

,count,median,mean,std
edutype,,,,
autodidacta,251,55000.0,65944.565737,46051.166693
escuela,1241,48000.0,55999.743755,40738.435273
trabajo,124,39760.0,44434.758065,24358.979047
online,258,38000.0,46389.496124,35644.238535
bootcamp,47,32000.0,33185.063830,18079.396810


Vemos que cursos online y bootcamps aparecen con los salarios más bajos. Pero eso puede ser engañoso, porque son opciones relativamente recientes y por lo tanto la gente que aprendió de esta manera no tiene tanta experiencia (que junto con el nivel de inglés es el factor que más influye en el salario). Así que ahora tomemos en cuenta solamente a los que tienen 5 años o menos de experiencia.

In [29]:
edutype = df[(df['experience']<=5)].groupby("edutype")["salarymx"].agg(['count', 'median', 'mean', 'std']).sort_values(by=['median'], ascending=False)
edutype

,count,median,mean,std
edutype,,,,
trabajo,26,35000.0,39742.461538,20236.183769
autodidacta,53,30000.0,38548.132075,31968.898563
online,137,30000.0,34084.992701,21520.537738
escuela,355,29000.0,32527.811268,20062.463554
bootcamp,37,24000.0,29862.108108,17489.937786


In [30]:
edutype.to_csv("tablas/edutype_5.csv")

Podemos ver que las cosas se emparejan mucho más. Incluso podemos decir que donde aprendiste a programar no es un factor significativo para tu salario. En otras palbras, lo que importa es lo que sabes, no como lo aprendiste.

Aprovechando que estamos en esto, vamos a ver si está cambiando la donde aprenden a programar las personas. Para ello, haremos un cruce de donde aprendieron vs años de experiencia.

In [31]:
exp_bin = df.groupby(['edutype','exp_bin'])['salarymx'].agg(['median', 'count']).fillna(0)
edutype_pct = exp_bin.groupby(level=1)['count'].apply(lambda x: x / float(x.sum()))
edutype_pct.head(20)

edutype      exp_bin     
autodidacta  (-0.1, 2.0]     0.078341
             (2.0, 4.0]      0.073359
             (4.0, 6.0]      0.127490
             (6.0, 8.0]      0.132275
             (8.0, 10.0]     0.100877
             (10.0, 14.0]    0.158491
             (14.0, 20.0]    0.176080
             (20.0, 40.0]    0.189573
bootcamp     (-0.1, 2.0]     0.096774
             (2.0, 4.0]      0.042471
             (4.0, 6.0]      0.023904
             (6.0, 8.0]      0.000000
             (8.0, 10.0]     0.030702
             (10.0, 14.0]    0.003774
             (14.0, 20.0]    0.003322
             (20.0, 40.0]    0.000000
escuela      (-0.1, 2.0]     0.557604
             (2.0, 4.0]      0.621622
             (4.0, 6.0]      0.605578
             (6.0, 8.0]      0.693122
Name: count, dtype: float64

In [32]:
data = {
    'exp': exp_labels,
    'escuela' : list(edutype_pct.xs('escuela')),
    'autodidacta': list(edutype_pct.xs('autodidacta')),
    'trabajo' : list(edutype_pct.xs('trabajo')),
    'online' : list(edutype_pct.xs('online')),
    'bootcamp' : list(edutype_pct.xs('bootcamp'))
}

edudf = pd.DataFrame(data)
edudf.sort_index(ascending=False, inplace=True)
edudf.head(20)

,exp,escuela,autodidacta,trabajo,online,bootcamp
7,20+,0.672986,0.189573,0.066351,0.071090,0.000000
6,15-20,0.697674,0.176080,0.066445,0.056478,0.003322
5,11-14,0.690566,0.158491,0.079245,0.067925,0.003774
4,9-10,0.618421,0.100877,0.092105,0.157895,0.030702
3,7-8,0.693122,0.132275,0.074074,0.100529,0.000000
2,5-6,0.605578,0.127490,0.067729,0.175299,0.023904
1,3-4,0.621622,0.073359,0.042471,0.220077,0.042471
0,0-2,0.557604,0.078341,0.027650,0.239631,0.096774


In [33]:


src = ColumnDataSource(edudf)
col_names = src.column_names

p = figure(x_range=exp_labels[::-1], plot_height=400, plot_width=600)

for col_name, color in zip(list(edudf.columns), Dark2[6]):
    if col_name == 'exp':
        continue

    p.add_tools(HoverTool(
        renderers= [p.line('exp', col_name, source=src, color=color, legend_label=col_name)],
        tooltips=[
            ('Formación', f'{col_name}'),
            ('Pct', '@'+col_name+'{0%}')
        ]
    ))

p.title.text = 'Evolución de donde aprendemos a programar'
p.xaxis.axis_label = 'Experiencia (años)'
p.yaxis.axis_label = 'Porcentaje'
p.yaxis.formatter = NumeralTickFormatter(format='0%')    
p.legend.location = (400,120)
    
show(p)


Llama la atención el crecimiento de los cursos online.

## Cliente local vs internacional

In [34]:
table = df.groupby("remote")["salarymx"].agg(['count', 'median', 'mean', 'std']).sort_values(by=['median'], ascending=False)
table.head(20)

,count,median,mean,std
remote,,,,
Y,863,60000.0,66251.811124,40216.536771
N,1058,35000.0,45284.076560,37494.672170


In [35]:
table = df.groupby("orgtype")["salarymx"].agg(['count', 'median', 'mean', 'std']).sort_values(by=['median'], ascending=False)
table.head(20)

,count,median,mean,std
orgtype,,,,
startup,190,60000.0,72188.010526,50303.858166
isv,292,53000.0,63006.291096,46804.263141
itservices,871,50000.0,54988.699196,34372.740791
corp,392,41000.0,49269.112245,38576.093546
gobierno,82,27000.0,31064.817073,19783.834582
uni,79,24000.0,31200.797468,23756.368652
freelance,15,16000.0,50098.666667,87943.831252


## Covid-19

In [36]:
covid = df[["covid_carga","covid_remoto", "covid_salario", "covid_apoyo"]].copy()
covid.value_counts(["covid_remoto"])




covid_remoto  
remote            1555
semipresencial     190
onsite             176
dtype: int64

## Special cases

In [37]:
segment = df[(df["city"] == "Guadalajara") & (df["experience"]>6)]
ingles = segment.groupby("english_label")["salarymx"].agg(['count', 'median', 'mean', 'std']).sort_values(by=['median'], ascending=False)
ingles.head(20)


,count,median,mean,std
english_label,,,,
Nativo o bilingue (ILR 5),11,92000.0,98727.272727,37029.963298
Avanzado: Puedo conversar y escribir sin problemas sobre cualquier tema (ILR 4),57,90000.0,95427.210526,37690.027835
Profesional: Puedo interactuar profesionalmente con colegas y clientes (ILR 3),54,60000.0,66057.407407,26547.555065
Limitado: Me doy a entender pero con errores de gramática (ILR 2),21,40000.0,51714.285714,29809.315418
Elemental: Sé lo básico para sobrevivir (ILR 1),7,24000.0,33428.571429,27232.683445


In [38]:
ingles.to_csv("tablas/gdl_ingles.csv")